<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> Simulación de procesos financieros. 

**Nombres:** Aranzazú Rendón Gómez, Héctor Daniel Chavez Orozco.
    
**Fecha:** 07 de mayo del 2021.

**Expediente:** 722272, 713442.
**Profesor:** Oscar David Jaramillo Zuluaga.

# Tarea 10: Clase 23
    
Liga del repositorio: https://github.com/Aranzazu-R/Tarea9y10_HectorChavez_AranzazuRendon.git

# Enunciado Tarea

Implementar el método de esquemas del trapecio, para valuar la opción call y put asiática con precio inicial, $S_0 = 100$, precio de ejercicio $K = 100$, tasa libre de riesgo $r = 0.10$, volatilidad $\sigma = 0.20$ y $T = 1$ año. Cuyo precio es $\approx 7.04$. Realizar la simulación en base a la siguiente tabla:
![imagen.png](tabla_tarea10.png)

Observe que en esta tabla se encuentran los intervalos de confianza de la aproximación obtenida y además el tiempo de simulación que tarda en encontrar la respuesta cada método. 
- Se debe entonces realizar una simulación para la misma cantidad de trayectorias y número de pasos y construir una Dataframe de pandas para reportar todos los resultados obtenidos.**(70 puntos)**
- Compare los resultados obtenidos con los resultados arrojados por la función `Riemann_approach`. Concluya. **(30 puntos)**

Se habilitará un enlace en canvas donde se adjuntará los resultados de dicha tarea

>**Nota:** Para generar índices de manera como se especifica en la tabla referirse a:
> - https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html
> - https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html
> - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.html



## RESPUESTA HECTOR

In [1]:
#importar los paquetes que se van a usar
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
from time import time
%matplotlib inline
#algunas opciones para Pandas
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 9)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 78)
pd.set_option('precision', 3)

In [2]:
def BSprices(mu,sigma,S0,NbTraj,NbStep):
    """
    Expresión de la solución de la ecuación de Black-Scholes
    St = S0*exp((r-sigma^2/2)*t+ sigma*DeltaW)
    
    Parámetros
    ---------
    mu    : Tasa libre de riesgo
    sigma : Desviación estándar de los rendimientos
    S0    : Precio inicial del activo subyacente
    NbTraj: Cantidad de trayectorias a simular
    NbStep: Número de días a simular
    """
    # Datos para la fórmula de St
    nu = mu-(sigma**2)/2
    DeltaT = 1/NbStep
    SqDeltaT = np.sqrt(DeltaT)
    DeltaW = SqDeltaT*np.random.randn(NbTraj,NbStep-1)
    
    # Se obtiene --> Ln St = Ln S0+ nu*DeltaT + sigma*DeltaW
    increments = nu*DeltaT + sigma*DeltaW
    concat = np.concatenate((np.log(S0)*np.ones([NbTraj,1]),increments),axis=1)
    
    # Se utiliza cumsum por que se quiere simular los precios iniciando desde S0
    LogSt = np.cumsum(concat,axis=1)
    # Se obtienen los precios simulados para los NbStep fijados
    St = np.exp(LogSt)
    # Vector con la cantidad de días simulados
    t = np.arange(0,NbStep)

    return St.T,t

def calc_daily_ret(closes):
    return np.log(closes/closes.shift(1)).iloc[1:]

## Sumas de Riemann

$\hat V_0^{(1)}= {e^{-rT} \over M} \sum_{j=1}^{M} \Bigg({1\over N} \sum_{i=0}^{N-1} S_{t_i}-K \Bigg)_+$

In [3]:
#Para la posicion call

def Riemann_approach_c(K:'Strike price',r:'Tasa libre de riesgo',S0:'Precio inicial',
                     NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad',T:'Tiempo de cierre del contrato en años',
                    flag=None):
    # Resolvemos la ecuación de black scholes para obtener los precios
    t_i = time()
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    # Obtenemos los precios promedios
    Average_t = prices.expanding().mean()
    # Definimos el dataframe de strikes
    strike = pd.DataFrame(K*np.ones([NbStep,NbTraj]), index=t)
    # Calculamos el call de la opción según la formula obtenida para Sumas de Riemann
    call = pd.DataFrame({'Prima':np.exp(-r*T) \
                 *np.fmax(Average_t-strike,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
    # intervalos de confianza
    confianza = 0.95
    sigma_est = call.sem().Prima
    mean_est = call.iloc[-1].Prima
    i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
    t_f = time()
    t_e = t_f-t_i
    return call.iloc[-1].Prima,i1[0],i1[1],i1[1]-i1[0],t_e

#Para la posicion put

def Riemann_approach_p(K:'Strike price',r:'Tasa libre de riesgo',S0:'Precio inicial',
                     NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad',T:'Tiempo de cierre del contrato en años',
                    flag=None):
    # Resolvemos la ecuación de black scholes para obtener los precios
    t_i = time()
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    # Obtenemos los precios promedios
    Average_t = prices.expanding().mean()
    # Definimos el dataframe de strikes
    strike = pd.DataFrame(K*np.ones([NbStep,NbTraj]), index=t)
    # Calculamos el call de la opción según la formula obtenida para Sumas de Riemann
    put = pd.DataFrame({'Prima':np.exp(-r*T) \
                 *np.fmax(strike-Average_t,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
    # intervalos de confianza
    confianza = 0.95
    sigma_est = put.sem().Prima
    mean_est = put.iloc[-1].Prima
    i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
    t_f = time()
    t_e = t_f-t_i
    return put.iloc[-1].Prima,i1[0],i1[1],i1[1]-i1[0],t_e

In [4]:
NbTraj = [1000,5000,10000,50000,100000,500000,1000000] # Trayectorias montecarlo
NbStep = [10,50,100] # quiero que se hagan con 10,50 y 100

S0 = 100     # Precio inicial
r = 0.10     # Tasa libre de riesgo 
sigma = 0.2  # volatilidad
K = 100      # Strike price
T = 1        # Tiempo de cierre - años

M_Riem_call = list(map(lambda N_tra:list(map(lambda N_ste:Riemann_approach_c(K,r,S0,N_tra,N_ste,sigma,T),NbStep)), NbTraj))

M_Riem_put = list(map(lambda N_tra:list(map(lambda N_ste:Riemann_approach_p(K,r,S0,N_tra,N_ste,sigma,T),NbStep)), NbTraj))

In [5]:
# Visualización de datos 
filas = ['Nbtray = %i' %i for i in NbTraj]
col = ['NbStep = %i' %i for i in NbStep]
df = pd.DataFrame(index=filas,columns=col)
df.loc[:,:] = M_Riem_call
df

,NbStep = 10,NbStep = 50,NbStep = 100
Nbtray = 1000,"(6.389528571393821, 5.13265605255047, 7.646401...","(6.8501107631815685, 6.340108395848878, 7.3601...","(6.939838610184799, 6.580773488998285, 7.29890..."
Nbtray = 5000,"(6.581148207847333, 5.280246147626176, 7.88205...","(6.956700548575076, 6.436884214426547, 7.47651...","(6.965632352164691, 6.606335947605618, 7.32492..."
Nbtray = 10000,"(6.403558354060992, 5.135243562608812, 7.67187...","(6.828564109586479, 6.3198583891874955, 7.3372...","(7.1341422532710945, 6.765121811294274, 7.5031..."
Nbtray = 50000,"(6.436129402452287, 5.160280398696523, 7.71197...","(6.939836695148108, 6.422065778115423, 7.45760...","(6.9500829009246825, 6.5934035312067625, 7.306..."
Nbtray = 100000,"(6.419270341007975, 5.146472711779325, 7.69206...","(6.92096862219878, 6.403879686689194, 7.438057...","(6.951525517600001, 6.593263297059958, 7.30978..."
Nbtray = 500000,"(6.4155008164984, 5.143421725334242, 7.6875799...","(6.91278787424646, 6.396626033319445, 7.428949...","(6.990089978033676, 6.630196022832808, 7.34998..."
Nbtray = 1000000,"(6.427149554756681, 5.152589671529145, 7.70170...","(6.912684980575286, 6.39667844593304, 7.428691...","(6.986177893302463, 6.626253512361358, 7.34610..."


In [11]:
indx = pd.MultiIndex(levels=[NbTraj,NbStep],codes = [[0,0,0,1,1,1,2,2,2,3,3,3,4,4,4,5,5,5,6,6,6],[0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2]])
aprox = np.array([M_Riem_call[0][0][0],M_Riem_call[0][1][0],M_Riem_call[0][2][0],M_Riem_call[1][0][0],M_Riem_call[1][1][0],M_Riem_call[1][2][0],M_Riem_call[2][0][0],M_Riem_call[2][1][0],M_Riem_call[2][2][0],M_Riem_call[3][0][0],M_Riem_call[3][1][0],M_Riem_call[3][2][0],M_Riem_call[4][0][0],M_Riem_call[4][1][0],M_Riem_call[4][2][0],M_Riem_call[5][0][0],M_Riem_call[5][1][0],M_Riem_call[5][2][0],M_Riem_call[6][0][0],M_Riem_call[6][1][0],M_Riem_call[6][2][0]])
Linf = np.array([M_Riem_call[0][0][1],M_Riem_call[0][1][1],M_Riem_call[0][2][1],M_Riem_call[1][0][1],M_Riem_call[1][1][1],M_Riem_call[1][2][1],M_Riem_call[2][0][1],M_Riem_call[2][1][1],M_Riem_call[2][2][1],M_Riem_call[3][0][1],M_Riem_call[3][1][1],M_Riem_call[3][2][1],M_Riem_call[4][0][1],M_Riem_call[4][1][1],M_Riem_call[4][2][1],M_Riem_call[5][0][1],M_Riem_call[5][1][1],M_Riem_call[5][2][1],M_Riem_call[6][0][1],M_Riem_call[6][1][1],M_Riem_call[6][2][1]])
Lsup = np.array([M_Riem_call[0][0][2],M_Riem_call[0][1][2],M_Riem_call[0][2][2],M_Riem_call[1][0][2],M_Riem_call[1][1][2],M_Riem_call[1][2][2],M_Riem_call[2][0][2],M_Riem_call[2][1][2],M_Riem_call[2][2][2],M_Riem_call[3][0][2],M_Riem_call[3][1][2],M_Riem_call[3][2][2],M_Riem_call[4][0][2],M_Riem_call[4][1][2],M_Riem_call[4][2][2],M_Riem_call[5][0][2],M_Riem_call[5][1][2],M_Riem_call[5][2][2],M_Riem_call[6][0][2],M_Riem_call[6][1][2],M_Riem_call[6][2][2]])
Ldiff = np.array(Lsup-Linf)
tiempos = np.array([M_Riem_call[0][0][3],M_Riem_call[0][1][3],M_Riem_call[0][2][3],M_Riem_call[1][0][3],M_Riem_call[1][1][3],M_Riem_call[1][2][3],M_Riem_call[2][0][3],M_Riem_call[2][1][3],M_Riem_call[2][2][3],M_Riem_call[3][0][3],M_Riem_call[3][1][3],M_Riem_call[3][2][3],M_Riem_call[4][0][3],M_Riem_call[4][1][3],M_Riem_call[4][2][3],M_Riem_call[5][0][3],M_Riem_call[5][1][3],M_Riem_call[5][2][3],M_Riem_call[6][0][3],M_Riem_call[6][1][3],M_Riem_call[6][2][3]])
tabla_Riem_call = pd.DataFrame(index=indx,columns=['Aproximacion','L. Inf','L. Sup','Long. 95%','Tiempo (segs)'])
tabla_Riem_call.index.names = (['Tray Monte Carlo','Núm pasos en el tiempo'])
tabla_Riem_call['Aproximacion'] = aprox
tabla_Riem_call['L. Inf'] = Linf
tabla_Riem_call['L. Sup'] = Lsup
tabla_Riem_call['Long. 95%'] = Ldiff
tabla_Riem_call['Tiempo (segs)'] = np.round(tiempos,2)
tabla_Riem_call

Aproximacion  L. Inf  L. Sup  \
Tray Monte Carlo Núm pasos en el tiempo                                 
1000             10                             6.390   5.133   7.646   
                 50                             6.850   6.340   7.360   
                 100                            6.940   6.581   7.299   
5000             10                             6.581   5.280   7.882   
                 50                             6.957   6.437   7.477   
...                                               ...     ...     ...   
500000           50                             6.913   6.397   7.429   
                 100                            6.990   6.630   7.350   
1000000          10                             6.427   5.153   7.702   
                 50                             6.913   6.397   7.429   
                 100                            6.986   6.626   7.346   

                                         Long. 95%  Tiempo (segs)  
Tray Monte Carlo Núm pasos en el tiempo                            
1000             10                          2.514           2.51  
                 50                          1.020           1.02  
                 100                         0.718           0.72  
5000             10                          2.602           2.60  
                 50                          1.040           1.04  
...                                            ...            ...  
500000           50                          1.032           1.03  
                 100                         0.720           0.72  
1000000          10                          2.549           2.55  
                 50                          1.032           1.03  
                 100                         0.720           0.72  

[21 rows x 5 columns]

In [12]:
indx = pd.MultiIndex(levels=[NbTraj,NbStep],codes = [[0,0,0,1,1,1,2,2,2,3,3,3,4,4,4,5,5,5,6,6,6],[0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2]])
aprox_Riem_put = np.array([M_Riem_put[0][0][0],M_Riem_put[0][1][0],M_Riem_put[0][2][0],M_Riem_put[1][0][0],M_Riem_put[1][1][0],M_Riem_put[1][2][0],M_Riem_put[2][0][0],M_Riem_put[2][1][0],M_Riem_put[2][2][0],M_Riem_put[3][0][0],M_Riem_put[3][1][0],M_Riem_put[3][2][0],M_Riem_put[4][0][0],M_Riem_put[4][1][0],M_Riem_put[4][2][0],M_Riem_put[5][0][0],M_Riem_put[5][1][0],M_Riem_put[5][2][0],M_Riem_put[6][0][0],M_Riem_put[6][1][0],M_Riem_put[6][2][0]])
Linf_Riem_put= np.array([M_Riem_put[0][0][1],M_Riem_put[0][1][1],M_Riem_put[0][2][1],M_Riem_put[1][0][1],M_Riem_put[1][1][1],M_Riem_put[1][2][1],M_Riem_put[2][0][1],M_Riem_put[2][1][1],M_Riem_put[2][2][1],M_Riem_put[3][0][1],M_Riem_put[3][1][1],M_Riem_put[3][2][1],M_Riem_put[4][0][1],M_Riem_put[4][1][1],M_Riem_put[4][2][1],M_Riem_put[5][0][1],M_Riem_put[5][1][1],M_Riem_put[5][2][1],M_Riem_put[6][0][1],M_Riem_put[6][1][1],M_Riem_put[6][2][1]])
Lsup_Riem_put = np.array([M_Riem_put[0][0][2],M_Riem_put[0][1][2],M_Riem_put[0][2][2],M_Riem_put[1][0][2],M_Riem_put[1][1][2],M_Riem_put[1][2][2],M_Riem_put[2][0][2],M_Riem_put[2][1][2],M_Riem_put[2][2][2],M_Riem_put[3][0][2],M_Riem_put[3][1][2],M_Riem_put[3][2][2],M_Riem_put[4][0][2],M_Riem_put[4][1][2],M_Riem_put[4][2][2],M_Riem_put[5][0][2],M_Riem_put[5][1][2],M_Riem_put[5][2][2],M_Riem_put[6][0][2],M_Riem_put[6][1][2],M_Riem_put[6][2][2]])
Ldiff_Riem_put = np.array(Lsup_Riem_put-Linf_Riem_put)
tiempos_Riem_put = np.array([M_Riem_put[0][0][3],M_Riem_put[0][1][3],M_Riem_put[0][2][3],M_Riem_put[1][0][3],M_Riem_put[1][1][3],M_Riem_put[1][2][3],M_Riem_put[2][0][3],M_Riem_put[2][1][3],M_Riem_put[2][2][3],M_Riem_put[3][0][3],M_Riem_put[3][1][3],M_Riem_put[3][2][3],M_Riem_put[4][0][3],M_Riem_put[4][1][3],M_Riem_put[4][2][3],M_Riem_put[5][0][3],M_Riem_put[5][1][3],M_Riem_put[5][2][3],M_Riem_put[6][0][3],M_Riem_put[6][1][3],M_Riem_put[6][2][3]])
tabla_Riem_put = pd.DataFrame(index=indx,columns=['Aproximacion','L. Inf','L. Sup','Long. 95%','Tiempo (segs)'])
tabla_Riem_put.index.names = (['Tray Monte Carlo','Núm pasos en el tiempo'])
tabla_Riem_put['Aproximacion'] = aprox_Riem_put
tabla_Riem_put['L. Inf'] = Linf_Riem_put
tabla_Riem_put['L. Sup'] = Lsup_Riem_put
tabla_Riem_put['Long. 95%'] = Ldiff_Riem_put
tabla_Riem_put['Tiempo (segs)'] = np.round(tiempos_Riem_put,2)
tabla_Riem_put

Aproximacion  L. Inf  L. Sup  \
Tray Monte Carlo Núm pasos en el tiempo                                 
1000             10                             2.218   1.788   2.648   
                 50                             2.167   2.040   2.294   
                 100                            2.345   2.249   2.441   
5000             10                             2.274   1.837   2.710   
                 50                             2.295   2.151   2.439   
...                                               ...     ...     ...   
500000           50                             2.342   2.193   2.491   
                 100                            2.347   2.247   2.447   
1000000          10                             2.218   1.793   2.643   
                 50                             2.327   2.180   2.475   
                 100                            2.352   2.252   2.452   

                                         Long. 95%  Tiempo (segs)  
Tray Monte Carlo Núm pasos en el tiempo                            
1000             10                          0.860           0.86  
                 50                          0.255           0.25  
                 100                         0.192           0.19  
5000             10                          0.873           0.87  
                 50                          0.288           0.29  
...                                            ...            ...  
500000           50                          0.297           0.30  
                 100                         0.199           0.20  
1000000          10                          0.850           0.85  
                 50                          0.295           0.30  
                 100                         0.200           0.20  

[21 rows x 5 columns]

# Esquema del trapecio
$\hat V_0^{(2)}= {e^{-rT} \over M} \sum_{j=1}^{M} \Bigg({h\over 2T} \sum_{i=0}^{N-1} S_{t_i}(2+rh+(W_{t_{i+1}}-W_{t_i})\sigma)-K \Bigg)_+$

In [13]:
def Trapecio_c(K:'Strike price',r:'Tasa libre de riesgo',S0:'Precio inicial',
                     NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad',T:'Tiempo de cierre del contrato en años',
                    flag=None):
    # Resolvemos la ecuación de black scholes para obtener los precios
    t_i = time()
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)

    h = T/NbStep
    Average_t = np.cumsum(np.multiply(prices,(2+r*h+np.random.randn(NbStep,NbTraj)*sigma)))*h/(2*T)
    # Definimos el dataframe de strikes
    strike = pd.DataFrame(K*np.ones([NbStep,NbTraj]), index=t)
    # Calculamos el call de la opción según la formula obtenida para Sumas de Riemann
    call = pd.DataFrame({'Prima':np.exp(-r*T) \
                 *np.fmax(Average_t-strike,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
    # intervalos de confianza
    confianza = 0.95
    sigma_est = call.sem().Prima
    mean_est = call.iloc[-1].Prima
    i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
    t_f = time()
    t_e = t_f-t_i
    return call.iloc[-1].Prima,i1[0],i1[1],i1[1]-i1[0],t_e


def Trapecio_p(K:'Strike price',r:'Tasa libre de riesgo',S0:'Precio inicial',
                     NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad',T:'Tiempo de cierre del contrato en años',
                    flag=None):
    # Resolvemos la ecuación de black scholes para obtener los precios
    t_i = time()
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
  
    h = T/NbStep
    Average_t = np.cumsum(np.multiply(prices,(2+r*h+np.random.randn(NbStep,NbTraj)*sigma)))*h/(2*T)
    # Definimos el dataframe de strikes
    strike = pd.DataFrame(K*np.ones([NbStep,NbTraj]), index=t)
    # Calculamos el put de la opción según la formula obtenida para Sumas de Riemann
    put = pd.DataFrame({'Prima':np.exp(-r*T) \
                 *np.fmax(strike-Average_t,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
    # intervalos de confianza
    confianza = 0.95
    sigma_est = put.sem().Prima
    mean_est = put.iloc[-1].Prima
    i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
    t_f = time()
    t_e = t_f-t_i
    return put.iloc[-1].Prima,i1[0],i1[1],i1[1]-i1[0],t_e

In [14]:
M_trap_call = list(map(lambda N_tra:list(map(lambda N_ste:Trapecio_c(K,r,S0,N_tra,N_ste,sigma,T),NbStep)), NbTraj))

M_trap_put = list(map(lambda N_tra:list(map(lambda N_ste:Trapecio_p(K,r,S0,N_tra,N_ste,sigma,T),NbStep)), NbTraj))

In [15]:
indx = pd.MultiIndex(levels=[NbTraj,NbStep],codes = [[0,0,0,1,1,1,2,2,2,3,3,3,4,4,4,5,5,5,6,6,6],[0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2]])
aprox_trap_call = np.array([M_trap_call[0][0][0],M_trap_call[0][1][0],M_trap_call[0][2][0],M_trap_call[1][0][0],M_trap_call[1][1][0],M_trap_call[1][2][0],M_trap_call[2][0][0],M_trap_call[2][1][0],M_trap_call[2][2][0],M_trap_call[3][0][0],M_trap_call[3][1][0],M_trap_call[3][2][0],M_trap_call[4][0][0],M_trap_call[4][1][0],M_trap_call[4][2][0],M_trap_call[5][0][0],M_trap_call[5][1][0],M_trap_call[5][2][0],M_trap_call[6][0][0],M_trap_call[6][1][0],M_trap_call[6][2][0]])
Linf_trap_call= np.array([M_trap_call[0][0][1],M_trap_call[0][1][1],M_trap_call[0][2][1],M_trap_call[1][0][1],M_trap_call[1][1][1],M_trap_call[1][2][1],M_trap_call[2][0][1],M_trap_call[2][1][1],M_trap_call[2][2][1],M_trap_call[3][0][1],M_trap_call[3][1][1],M_trap_call[3][2][1],M_trap_call[4][0][1],M_trap_call[4][1][1],M_trap_call[4][2][1],M_trap_call[5][0][1],M_trap_call[5][1][1],M_trap_call[5][2][1],M_trap_call[6][0][1],M_trap_call[6][1][1],M_trap_call[6][2][1]])
Lsup_trap_call = np.array([M_trap_call[0][0][2],M_trap_call[0][1][2],M_trap_call[0][2][2],M_trap_call[1][0][2],M_trap_call[1][1][2],M_trap_call[1][2][2],M_trap_call[2][0][2],M_trap_call[2][1][2],M_trap_call[2][2][2],M_trap_call[3][0][2],M_trap_call[3][1][2],M_trap_call[3][2][2],M_trap_call[4][0][2],M_trap_call[4][1][2],M_trap_call[4][2][2],M_trap_call[5][0][2],M_trap_call[5][1][2],M_trap_call[5][2][2],M_trap_call[6][0][2],M_trap_call[6][1][2],M_trap_call[6][2][2]])
Ldiff_trap_call = np.array(Lsup_trap_call-Linf_trap_call)
tiempos_trap_call = np.array([M_trap_call[0][0][3],M_trap_call[0][1][3],M_trap_call[0][2][3],M_trap_call[1][0][3],M_trap_call[1][1][3],M_trap_call[1][2][3],M_trap_call[2][0][3],M_trap_call[2][1][3],M_trap_call[2][2][3],M_trap_call[3][0][3],M_trap_call[3][1][3],M_trap_call[3][2][3],M_trap_call[4][0][3],M_trap_call[4][1][3],M_trap_call[4][2][3],M_trap_call[5][0][3],M_trap_call[5][1][3],M_trap_call[5][2][3],M_trap_call[6][0][3],M_trap_call[6][1][3],M_trap_call[6][2][3]])
tabla_trap_call = pd.DataFrame(index=indx,columns=['Aproximacion','L. Inf','L. Sup','Long. 95%','Tiempo (segs)'])
tabla_trap_call.index.names = (['Tray Monte Carlo','Núm pasos en el tiempo'])
tabla_trap_call['Aproximacion'] = aprox_trap_call
tabla_trap_call['L. Inf'] = Linf_trap_call
tabla_trap_call['L. Sup'] = Lsup_trap_call
tabla_trap_call['Long. 95%'] = Ldiff_trap_call
tabla_trap_call['Tiempo (segs)'] = np.round(tiempos_trap_call,2)
tabla_trap_call

Aproximacion  L. Inf  L. Sup  \
Tray Monte Carlo Núm pasos en el tiempo                                 
1000             10                             6.919   5.558   8.279   
                 50                             7.282   6.852   7.712   
                 100                            6.879   6.609   7.148   
5000             10                             6.808   5.472   8.144   
                 50                             7.144   6.722   7.566   
...                                               ...     ...     ...   
500000           50                             7.022   6.607   7.437   
                 100                            7.028   6.751   7.304   
1000000          10                             6.903   5.549   8.257   
                 50                             7.008   6.594   7.421   
                 100                            7.012   6.736   7.287   

                                         Long. 95%  Tiempo (segs)  
Tray Monte Carlo Núm pasos en el tiempo                            
1000             10                          2.722           2.72  
                 50                          0.860           0.86  
                 100                         0.539           0.54  
5000             10                          2.672           2.67  
                 50                          0.844           0.84  
...                                            ...            ...  
500000           50                          0.830           0.83  
                 100                         0.553           0.55  
1000000          10                          2.708           2.71  
                 50                          0.828           0.83  
                 100                         0.551           0.55  

[21 rows x 5 columns]

In [16]:
indx = pd.MultiIndex(levels=[NbTraj,NbStep],codes = [[0,0,0,1,1,1,2,2,2,3,3,3,4,4,4,5,5,5,6,6,6],[0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2]])
aprox_trap_put = np.array([M_trap_put[0][0][0],M_trap_put[0][1][0],M_trap_put[0][2][0],M_trap_put[1][0][0],M_trap_put[1][1][0],M_trap_put[1][2][0],M_trap_put[2][0][0],M_trap_put[2][1][0],M_trap_put[2][2][0],M_trap_put[3][0][0],M_trap_put[3][1][0],M_trap_put[3][2][0],M_trap_put[4][0][0],M_trap_put[4][1][0],M_trap_put[4][2][0],M_trap_put[5][0][0],M_trap_put[5][1][0],M_trap_put[5][2][0],M_trap_put[6][0][0],M_trap_put[6][1][0],M_trap_put[6][2][0]])
Linf_trap_put= np.array([M_trap_put[0][0][1],M_trap_put[0][1][1],M_trap_put[0][2][1],M_trap_put[1][0][1],M_trap_put[1][1][1],M_trap_put[1][2][1],M_trap_put[2][0][1],M_trap_put[2][1][1],M_trap_put[2][2][1],M_trap_put[3][0][1],M_trap_put[3][1][1],M_trap_put[3][2][1],M_trap_put[4][0][1],M_trap_put[4][1][1],M_trap_put[4][2][1],M_trap_put[5][0][1],M_trap_put[5][1][1],M_trap_put[5][2][1],M_trap_put[6][0][1],M_trap_put[6][1][1],M_trap_put[6][2][1]])
Lsup_trap_put = np.array([M_trap_put[0][0][2],M_trap_put[0][1][2],M_trap_put[0][2][2],M_trap_put[1][0][2],M_trap_put[1][1][2],M_trap_put[1][2][2],M_trap_put[2][0][2],M_trap_put[2][1][2],M_trap_put[2][2][2],M_trap_put[3][0][2],M_trap_put[3][1][2],M_trap_put[3][2][2],M_trap_put[4][0][2],M_trap_put[4][1][2],M_trap_put[4][2][2],M_trap_put[5][0][2],M_trap_put[5][1][2],M_trap_put[5][2][2],M_trap_put[6][0][2],M_trap_put[6][1][2],M_trap_put[6][2][2]])
Ldiff_trap_put = np.array(Lsup_trap_put-Linf_trap_put)
tiempos_trap_put = np.array([M_trap_put[0][0][3],M_trap_put[0][1][3],M_trap_put[0][2][3],M_trap_put[1][0][3],M_trap_put[1][1][3],M_trap_put[1][2][3],M_trap_put[2][0][3],M_trap_put[2][1][3],M_trap_put[2][2][3],M_trap_put[3][0][3],M_trap_put[3][1][3],M_trap_put[3][2][3],M_trap_put[4][0][3],M_trap_put[4][1][3],M_trap_put[4][2][3],M_trap_put[5][0][3],M_trap_put[5][1][3],M_trap_put[5][2][3],M_trap_put[6][0][3],M_trap_put[6][1][3],M_trap_put[6][2][3]])
tabla_trap_put = pd.DataFrame(index=indx,columns=['Aproximacion','L. Inf','L. Sup','Long. 95%','Tiempo (segs)'])
tabla_trap_put.index.names = (['Tray Monte Carlo','Núm pasos en el tiempo'])
tabla_trap_put['Aproximacion'] = aprox_trap_put
tabla_trap_put['L. Inf'] = Linf_trap_put
tabla_trap_put['L. Sup'] = Lsup_trap_put
tabla_trap_put['Long. 95%'] = Ldiff_trap_put
tabla_trap_put['Tiempo (segs)'] = np.round(tiempos_trap_put,2)
tabla_trap_put

Aproximacion  L. Inf  L. Sup  \
Tray Monte Carlo Núm pasos en el tiempo                                 
1000             10                             2.070 -15.102  19.242   
                 50                             2.432  -5.017   9.880   
                 100                            2.394  -2.870   7.658   
5000             10                             2.195 -14.927  19.316   
                 50                             2.296  -5.171   9.762   
...                                               ...     ...     ...   
500000           50                             2.332  -5.130   9.794   
                 100                            2.359  -2.898   7.616   
1000000          10                             2.231 -14.885  19.347   
                 50                             2.332  -5.130   9.794   
                 100                            2.351  -2.906   7.608   

                                         Long. 95%  Tiempo (segs)  
Tray Monte Carlo Núm pasos en el tiempo                            
1000             10                         34.344          34.34  
                 50                         14.897          14.90  
                 100                        10.528          10.53  
5000             10                         34.243          34.24  
                 50                         14.933          14.93  
...                                            ...            ...  
500000           50                         14.924          14.92  
                 100                        10.514          10.51  
1000000          10                         34.232          34.23  
                 50                         14.924          14.92  
                 100                        10.514          10.51  

[21 rows x 5 columns]

## RESPUESTA ARANZAZU